
# Simulation of the spread of SARS-CoV-2 with the SEIR model

The SEIR model uses a system of differential equations to simulate the pandemic. The Python module SciPy contains the functions 'odeint ()' and 'curve_fit ()' to calculate the SEIR model and to optimize some selected parameters so that they match the case numbers for Portugal reported by the John Hopkins University.

The simulation starts with a few imports:

In [1]:
from IPython.display import display, Markdown, SVG
from equations import SEIR
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import datetime as dt
import numpy as np
import pandas as pd
import altair as alt
import requests
import os

The implementation of the model is in the class `SEIR` in [equations.py] 

The following web service conveniently already provides current figures from John Hopkins University as JSON (the edition contains the latest four numbers):

In [2]:
portugal_json = requests.get("https://corona.ersatzworld.net/data/Portugal.json").json()
first_date = dt.datetime.strptime(portugal_json["first_date"], "%Y-%m-%d")
data = pd.DataFrame(np.stack([
        [first_date + dt.timedelta(days=i) for i in range(len(portugal_json["total"]))], 
        portugal_json["total"]
    ], axis=1), 
    columns=("Tag", "Number of cases"))
data.tail(4)


,Tag,Number of cases
106,2020-05-07,26715
107,2020-05-08,27268
108,2020-05-09,27406
109,2020-05-10,27581


The next step calculates the `SEIR` model and fits four parameters so that the model fits the data from` data` as closely as possible.

The simulation starts on March 2, 2020 (first line). On March 14, 2020 and March 20, 2020, exit restrictions and a contact block affected the numbers, so that the SEIR model assumed different values ​​for $ R_ {0} $ for the periods before, between and after these dates. These three values ​​are the first three parameters of optimization with `curve_fit ()`. As a fourth parameter, the optimization also determines the unknown value $ E_ {0} $, the number of infected but not yet infectious corona patients on March 2.

In [3]:
data = data[data["Tag"] >= dt.datetime(year=2020, month=2, day=21)].reset_index(drop=True)

it1 = dt.datetime(year=2020, month=3, day=20)
it2 = dt.datetime(year=2020, month=3, day=28)
it3 = dt.datetime(year=2020, month=4, day=26)
it4 = dt.datetime(year=2020, month=5, day=11)
it5 = dt.datetime(year=2020, month=8, day=1)
cases = np.array(data["Number of cases"])
bev_de = portugal_json["population"]
times = np.arange(0., len(cases), 1.0)
model = SEIR(p0=(bev_de-data["Number of cases"][0], 0, data["Number of cases"][0], 0),
             intervention_times=[(it1-data["Tag"][0]).days, 
                                 (it2-data["Tag"][0]).days, 
                                 (it3-data["Tag"][0]).days,
                                 (data["Tag"][len(cases)-1]-data["Tag"][0]).days, 
                                 (data["Tag"][len(cases)-1]-data["Tag"][0]).days],
             t_vals=times)

params, _ = curve_fit(
    model, 
    xdata=times, 
    ydata=cases,
    p0=[3.7, 2.7, 0.8, 0.8, 0.],
    bounds=(
        [0., 0., 0., 0., 0.],
        [10., 10., 10., 10., bev_de]
    )
)
display(Markdown("Result of the optimization with `curve_fit()`:\n\n" +
    " 1. $R_{0}$ " + "without restrictions: {0:.3f}".format(params[0]) + "\n\n" +
    " 2. $R_{0}$ " + "with exit restrictions: {0:.3f}".format(params[1]) + "\n\n" +
    " 3. $R_{0}$ " + "with contact lock: {0:.3f}".format(params[2]) + "\n\n" +
    " 4. $R_{0}$ " + "with relaxed contact lock (shops up to 800m² open): {0:.3f}".format(params[3]) + "\n\n" +             
    " 5. $E_{0}$" + ": {0:.1f}".format(params[4])))


Result of the optimization with `curve_fit()`:

 1. $R_{0}$ without restrictions: 1.683

 2. $R_{0}$ with exit restrictions: 2.867

 3. $R_{0}$ with contact lock: 1.075

 4. $R_{0}$ with relaxed contact lock (shops up to 800m² open): 0.645

 5. $E_{0}$: 135.3

With these parameters, the following code calculates the model's predictions for 5 days into the future:

In [4]:
r0, r1, r2, r3, e0 = params
prediction_days = 5
times = np.arange(0., len(cases)+prediction_days, 1.0)
seir_predictions = model.getSEIR(times, [r0, r1, r2, r3, r3, r3], e0)
sum_predictions = np.sum(seir_predictions[:, 1:], axis=1)

The number of "susceptible" people, who could still be infected but have not yet been infected, is very high at the beginning of the simulation (slightly less than the population). A diagram that shows 10 million and 100,000 with the same y-axis would only show the interesting case numbers very small above the x-axis. To prevent this from happening, the function `clip_values ()` sets high values to `None`:

In [5]:
def clip_values(v, max_val=bev_de):
    return np.where(np.less_equal(v, max_val), v, None)

In preparation for a diagram that shows how well the simulation fits the real data, the following lines pack the necessary data series one after the other into a Pandas `Data Frame`. Using the `` Type '' column, Alatair later assigns the colors of the individual curves in the diagram.

In [6]:
dates = [data["Tag"][0] + dt.timedelta(days=i) for i in times]
seir_data = pd.DataFrame(np.concatenate([
    np.concatenate([np.stack([dates, clip_values(seir_predictions[:, i], cases[-1]*1.2), [t]*len(times)], axis=1)
                    for i, t in enumerate(["Susceptible", "Exposed", "Infectious", "Removed"])]),
    np.stack([dates, clip_values(sum_predictions, cases[-1]*1.2), ["simulated sum"]*len(times)], axis=1)]),
    columns=("Tag", "Number of cases", "Type")
)
data["Type"] = "counted cases"
seir_data = seir_data.append(data, ignore_index=True)
seir_data

,Tag,Number of cases,Type
0,2020-02-21,None,Susceptible
1,2020-02-22,None,Susceptible
2,2020-02-23,None,Susceptible
3,2020-02-24,None,Susceptible
4,2020-02-25,None,Susceptible
...,...,...,...
500,2020-05-06,26182,counted cases
501,2020-05-07,26715,counted cases
502,2020-05-08,27268,counted cases
503,2020-05-09,27406,counted cases


The following function creates a diagram with several levels: The curves and the legend draw `line`. The `selection`` nearest` uses the interactive diagram to select the day that best fits the position of the mouse. `points` draws little squiggles over the values of the curve on that day. `text` supplements the numerical value of the curled values. `rules` draws a gray line that highlights the day and` date_text` writes out the day as text at the top of the diagram. The function `alt.layer ()` combines all these elements in a diagram.

In addition to the data frame, the `plot ()` function also accepts optional parameters for the position of the legend and the dimensions of the entire diagram. The notebook later uses them for other diagrams with different scales.

In [7]:
def plot(data, legendX=20, legendY=20, width=830, height=400):
    line = alt.Chart(data).mark_line(point=False).encode(
        alt.X("Tag", title="Tag"),
        alt.Y("Number of cases:Q", title="Number of Cases"),
        color=alt.Color("Type:N", 
                        scale=alt.Scale(scheme="dark2"),
                        legend=alt.Legend(
            orient="none", legendX=legendX, legendY=legendY,
            fillColor="white", strokeColor="black", cornerRadius=7, padding=6,
            title="Groups"))
    )
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=['Tag'], empty='none')
    selectors = alt.Chart(data).mark_point().encode(
        alt.X("Tag", title="Tag"),
        opacity=alt.value(0),
    ).add_selection(nearest)
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )
    text = line.mark_text(align='left', dx=4, dy=-8).encode(
        text=alt.condition(nearest, 'Number of cases:Q', alt.value(' '))
    )
    rules = alt.Chart(data).mark_rule(color='gray').encode(
        alt.X("Tag", title="Tag"),
    ).transform_filter(nearest)
    date_text = rules.mark_text(align='left', dx=4, dy=-8).encode(
        y=alt.value(16),
        color=alt.value('gray'),
        text=alt.condition(nearest, 'Tag', alt.value(' '))
    )
    return alt.layer(
        line, selectors, points, rules, text, date_text
    ).properties(width=width, height=height)
  
chart = plot(seir_data)
chart

alt.LayerChart(...)

In [8]:
def save_and_display(chart, filename):
    chart.save(os.path.join("charts", filename + ".svg"))
    chart.save(os.path.join("charts", filename + ".png"))
    chart.save(os.path.join("charts", filename + ".html"))
    with open(os.path.join("charts", filename + ".svg"), 'r') as f:
        display(SVG(f.read()))

#save_and_display(chart, "fit-quality")

It is easy to see in the diagram that the model follows the real numbers with small deviations.

On this basis, you can now calculate a forecast for the next 800 days:

In [9]:
def prediction(prediction_days=365, max_val=bev_de, r4=r3, r5=r3,
               it1 = dt.datetime(year=2020, month=3, day=14),
               it2 = dt.datetime(year=2020, month=3, day=20),
               it3 = dt.datetime(year=2020, month=4, day=27),
               it4 = dt.datetime(year=2020, month=5, day=4),
               it5 = dt.datetime(year=2020, month=5, day=4)):
    cases = np.array(data["Number of cases"])
    times = np.arange(0., len(cases), 1.0)
    model = SEIR(p0=(bev_de-data["Number of cases"][0], 0, data["Number of cases"][0], 0),
                 intervention_times=[(it1-data["Tag"][0]).days, 
                                     (it2-data["Tag"][0]).days, 
                                     (it3-data["Tag"][0]).days, 
                                     (it4-data["Tag"][0]).days,
                                     (it5-data["Tag"][0]).days],
                 t_vals=times)
    times = np.arange(0., len(cases)+prediction_days, 1.0)
    seir_predictions = model.getSEIR(times, [r0, r1, r2, r3, r4, r5], e0)
    sum_predictions = np.sum(seir_predictions[:, 1:], axis=1)
    dates = [data["Tag"][0] + dt.timedelta(days=i) for i in times]
    seir_data = pd.DataFrame(np.concatenate([
        np.concatenate([np.stack([dates, clip_values(seir_predictions[:, i], max_val), [t]*len(times)], axis=1)
                        for i, t in enumerate(["Susceptible", "Exposed", "Infectious", "Removed"])]),
        np.stack([dates, clip_values(sum_predictions, max_val), ["simuleted sum"]*len(times)], axis=1)]),
        columns=("Tag", "Number of cases", "Type")
    )
    data["Type"] = "counted cases"
    return seir_data.append(data, ignore_index=True)

chart = plot(prediction(prediction_days=800), legendY=50, height=300)
chart

alt.LayerChart(...)

In [10]:
#save_and_display(chart, "800_days")


The flat curve shows that there will be few simultaneous infections thanks to the current restrictions. The health care system will cope with such a flat peak.

The curve will be more spectacular if the restrictions would end on May 3, increasing $ R_ {0} $ to before March 20:

In [11]:
chart = plot(prediction(prediction_days=100, r4=r0, r5=r0), legendY=50, height=225)
chart

alt.LayerChart(...)

In [12]:
#save_and_display(chart, "second_peak")

If you restrict the values a bit, the enormous peak becomes visible:

In [13]:
chart = plot(prediction(prediction_days=95, max_val=12000000, r4=r0, r5=r0))
chart

alt.LayerChart(...)

In [14]:
#save_and_display(chart, "second_peak_detail")

Now for a few calculation models to play around with: `it4` and` it5` are two days on which measures are relaxed. We preset January 1, 2021, from which only exit restrictions apply. This means `r4 = r1`, since the reproduction number $ R_ {0} $ climbs back to the value before the contact block as soon as it is released. Since the spread then continues to be slowed down by the initial restrictions, a second outbreak results, which does not increase as steeply as without restrictions.

On February 20, 2021 (`it5`), the initial restrictions will also be lifted in our default setting. $ R_ {0} $ therefore rises again to the value before all measures, so `r5 = r0`.

In addition, you can use `prediction_days` to set how far the simulation calculates in the future. `max_val` hides values ​​that are too high in the diagram, so that the y-axis does not have to display a value calculation that is too large and smaller curves can be better recognized.

Feel free to play around with the values behind the comments and create your own scenarios for relaxing the restrictions:

In [15]:
# Day of the removal of the contact block
it4 = dt.datetime(year=2021, month=1, day=1)

# Expected number of reproductions after removal of the contact block
r4 = r1


# Day of lifting of exit restrictions
it5 = dt.datetime(year=2021, month=2, day=20)

# Expected number of reproductions after removal of the contact block
r5 = r0


# Number of days the simulation calculates in the future
prediction_days=500

# Highest value in the diagram (so that the y-axis does not compress the interesting values ​​too much)
max_val=12000000

chart = plot(prediction(prediction_days=prediction_days, max_val=max_val, r4=r4, r5=r5, it4=it4, it5=it5))
chart

alt.LayerChart(...)

In [16]:
#save_and_display(chart, "loose-restrictions")

For an interesting variant of the simulation, just set $ R_ {0} $ to values that were not calculated by `curve_fit ()`. For example, after the intervention days, values for $ R_ {0} $ that are less than 1 ensure that the peak is over on this day. For example, very small $ R_ {0} $ would provide accurate tracking and isolation of the infected. The following diagram is purely hypothetical because of the invented values:

In [17]:
# Day of the removal of the contact block
it4 = dt.datetime(year=2020, month=7, day=1)

# Expected number of reproductions after removal of the contact block
r4 = 0.99


# Day of lifting of exit restrictions
it5 = dt.datetime(year=2020, month=9, day=1)
# Expected number of reproductions after removal of the contact block
r5 = 0.75


# Number of days the simulation calculates in the future
prediction_days=150

# Highest value in the diagram (so that the y-axis does not compress the interesting values ​​too much)
max_val=800000

chart = plot(prediction(prediction_days=prediction_days, max_val=max_val, r4=r4, r5=r5, it4=it4, it5=it5))
chart

alt.LayerChart(...)

In [18]:
#save_and_display(chart, "dreamed-infection-rates")